In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [3]:
# 设置随机种子以便结果可复现
torch.manual_seed(1)
np.random.seed(1)

# 生成训练数据集和标签
def generate_data(num_samples, noise=0.1):
    X = np.random.rand(num_samples, 100)
    # 标签为输入的前10个数的和加一些随机噪音
    y = np.sum(X[:, :10], axis=1) + np.random.normal(0, noise, num_samples)
    return X, y

In [4]:
X_train, y_train = generate_data(50, noise=0.01)
X_test, y_test = generate_data(100, noise=0)

# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [45]:
# 定义模型
class myDNN(nn.Module):
    def __init__(self):
        super(myDNN, self).__init__()
        
        self.fc = nn.Linear(100, 1)
        # self.fc = nn.Sequential(
        #     nn.Linear(100, 100),
        #     nn.ReLU(),
        #     nn.Linear(100, 1)
        # )

        # nn.init.normal_(self.linear.weight, 0, 0.1)

    def forward(self, x):
        out = self.fc(x)
        return out
    

class myCNN(nn.Module):
    def __init__(self):
        super(myCNN, self).__init__()

        # 使用1维卷积对x进行卷积，其中x为100维向量
        self.conv1 = nn.Conv1d(1, 1, 5, 5, 0)

        # self.fc = nn.Sequential(
        #     nn.Linear(10, 100),
        #     nn.ReLU(),
        #     nn.Linear(100, 1)
        #     # nn.Linear(10, 1)
        # )

        self.fc = nn.Linear(20, 1)

    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.conv1(x)
        out = out.view(-1, 20)
        out = self.fc(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
def train(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor):
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)

    # 训练模型
    num_epochs = 10000

    train_loss_list = []
    test_loss_list = []

    for epoch in range(num_epochs):

        model.train()
        # 前向传播
        outputs = model(X_train_tensor.to(device))
        train_loss = criterion(outputs, y_train_tensor.to(device))

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        scheduler.step()
        
        # 测试模型
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test_tensor.to(device))
            test_loss = criterion(y_pred, y_test_tensor.to(device))
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.2e}, Test Loss: {test_loss:.2e}', end='\r')

        train_loss_list.append(train_loss.item())
        test_loss_list.append(test_loss.item())

    return train_loss_list, test_loss_list, model

In [39]:
dnn = myDNN().to(device)
dnn_train_loss, dnn_test_loss, dnn = train(dnn, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

In [46]:
cnn = myCNN().to(device)
cnn_train_loss, cnn_test_loss, cnn = train(cnn, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

In [41]:
# 画loss图

fig = plt.figure(figsize=(8, 5))
format_settings(fs=18, left=0.15, right=0.95)
n = 10000
plt.loglog(dnn_train_loss[:n], label='DNN Train Loss', c='#C82423')
plt.loglog(dnn_test_loss[:n], label='DNN Test Loss', c='#C82423', ls='--')
plt.loglog(cnn_train_loss[:n], label='conv + DNN Train Loss', c='#2878B5')
plt.loglog(cnn_test_loss[:n], label='conv + DNN Test Loss', c='#2878B5', ls='--')
# plt.xlim([0, 10000])

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('dnn_vs_conv.png', dpi=200)
plt.close()

In [10]:
# 用criterion直接计算Y与X前10位和的loss
Z = np.sum(X_train[:, :10], axis=1)
Z = torch.tensor(Z, dtype=torch.float32).unsqueeze(1)
loss = criterion(Z, y_train_tensor)
print(f'Loss of Y and X\'s first 9 elements sum: {loss:.2e}')

Loss of Y and X's first 9 elements sum: 0.00e+00
